In [1]:
import torch
from transformers import GPT2LMHeadModel
from transformers import PreTrainedTokenizerFast
import os
import re
from tqdm import tqdm
import argparse
import subprocess
# import gluonnlp
import torch
from torch.utils.data import DataLoader 
from gluonnlp.data import SentencepieceTokenizer 

# tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")
# https://hipgyung.tistory.com/entry/%ED%95%9C%EA%B5%AD%EC%96%B4-%EA%B0%80%EC%82%AC-%EB%8D%B0%EC%9D%B4%ED%84%B0-KoGPT2-Fine-Tuning
# https://github.com/gyunggyung/KoGPT2-FineTuning
# https://ratsgo.github.io/nlpbook/docs/generation/inference2/

# git clone https://github.com/SKT-AI/KoGPT2.git
# cd KoGPT2
# pip install -r requirements.txt
# pip install .
# from kogpt2.utils import get_tokenizer
# from kogpt2.utils import download, tokenizer
# from kogpt2.model.torch_gpt2 import GPT2Config, GPT2LMHeadModel
# from kogpt2.data import Read_Dataset
# from kogpt2.model.sample import sample_sequence

# https://github.com/haven-jeon/KoGPT2-chatbot/blob/master/train_torch.py

In [2]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [3]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(pretrained_model_name_or_path="skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>', sep_token="<sep>", cls_token="<cls>", additional_special_tokens=["test", "<turn>"]) 

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
text = '근육이 커진다 그렇다면 얼마나 커질까?'
input_ids = tokenizer.encode(text)

gen_ids = model.generate(torch.tensor([input_ids]),
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)

In [8]:
input_ids

[33245, 10114, 8265, 7182, 24042, 15570, 45282, 47804]

In [6]:
gen_ids

tensor([[33245, 10114,  8265,  7182, 24042, 15570, 45282, 47804, 16563,   377,
          6947,  7406, 10846,  9022,  6855, 46651,  8234,   406,  9162, 12928,
          9782,  9018, 18814,  9148, 47041, 14226, 11649,  9659,  7991,  9337,
          6969,  8084,   376,  9394,  9863, 23066,  7283,  9745,  9736, 11064,
          9080,  9548,  9290,  9054, 14675, 20192, 49421,  9198,  6958, 11357,
         12201,   387, 11132, 15605, 15842, 23475,  9025, 13972,  9318,  7172,
         14364,  9037,  7601,   389, 10351, 12857, 11793, 17582, 13023,  9277,
         10578,  9432,  9098,  7071,  9348,  9078,  7801, 25856, 27747,  9094,
         19802, 12687,  9267, 19933,  8267, 23498, 12521, 11403, 12102,  9427,
          9036,  9515, 10917,  9135,  8718, 16691,  9194,  7194, 11991, 24692,
          9207, 22375,  9676, 20289, 12487, 13358,  8263,  9111, 16650, 11777,
         10021, 19520, 17932,  9320,  8052, 14662, 50082,  8146, 10431, 19935,
         13875,  9258, 13701, 37993, 10138, 10899, 4

In [7]:
tokenizer.batch_decode(gen_ids.tolist())

['근육이 커진다 그렇다면 얼마나 커질까?"\n"그렇다면 그건 뭐죠? 당신은 지금 이 순간부터 당신의 몸을 어떻게 만들어야 할까요! 그리고 어떤 식으로든 자신의 몸 안에 있는 모든 것을 다 없애야만 합니다. 그러기 위해서는 우선, 자신이 원하는 대로 움직일 수 있어야 합니다, 제발. 하지만 이제부터는 내가 해야 하는 일을 모두 해내도록 하세요. 그러면 내 몸은 점점 더 작아질 겁니다. 그래서 나는 다시 한 번 이렇게 말했습니다. \'당신이 나를 위해 무엇을 하고 있는지 알고 싶지 않다면, 그것은 바로 나의 몸이 아니오\'라고."\n이렇게 말하고 나서 그는 잠시 생각에 잠겼다.\n그리고는']

In [65]:
tokenizer.batch_decode(gen_ids.tolist())

['근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.\n특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.\n또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.\n아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.\n운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.\n운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.\n운동을']

In [14]:
input_ids

[1, 33245, 10114, 12748, 11357, 739, 3]

In [15]:
tokenizer.decode(input_ids)

'</s> 근육이 커지기 위해서는 <pad>'

In [ ]:
https://github.com/haven-jeon/KoGPT2-chatbot/blob/master/train_torch.py

In [ ]:
class CharDataset(Dataset):
    def __init__(self, chats, max_len=32):
        self._data = chats
        self.first = True
        self.q_token = U_TKN
        self.a_token = S_TKN
        self.sent_token = SENT
        self.bos = BOS
        self.eos = EOS
        self.mask = MASK
        self.pad = PAD
        self.max_len = max_len
        self.tokenizer = TOKENIZER 

    def __len__(self):
        return len(self._data)

    def __getitem__(self, idx):
        turn = self._data.iloc[idx]
        q = turn['Q']
        a = turn['A']
        sentiment = str(turn['label'])
        q_toked = self.tokenizer.tokenize(self.q_token + q + \
                                          self.sent_token + sentiment)   
        q_len = len(q_toked)
        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len
            if a_len <= 0:
                q_toked = q_toked[-(int(self.max_len/2)):]
                q_len = len(q_toked)
                a_len = self.max_len - q_len
                assert a_len > 0
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)
            assert a_len == len(a_toked), f'{a_len} ==? {len(a_toked)}'
        # [mask, mask, ...., mask, ..., <bos>,..A.. <eos>, <pad>....]
        labels = [
            self.mask,
        ] * q_len + a_toked[1:]
        if self.first:
            logging.info("contexts : {}".format(q))
            logging.info("toked ctx: {}".format(q_toked))
            logging.info("response : {}".format(a))
            logging.info("toked response : {}".format(a_toked))
            logging.info('labels {}'.format(labels))
            self.first = False
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        self.max_len
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]
        return(token_ids, np.array(mask),
               labels_ids)

In [2]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>') 
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:


parser = argparse.ArgumentParser()
parser.add_argument('--epoch', type=int, default=200,
					help="epoch 를 통해서 학습 범위를 조절합니다.")
parser.add_argument('--save_path', type=str, default='./checkpoint/',
					help="학습 결과를 저장하는 경로입니다.")
parser.add_argument('--load_path', type=str, default='./checkpoint/Alls/KoGPT2_checkpoint_296000.tar', #
					help="학습된 결과를 불러오는 경로입니다.")
parser.add_argument('--samples', type=str, default="samples/",
					help="생성 결과를 저장할 경로입니다.")
parser.add_argument('--data_file_path', type=str, default='dataset/lyrics_dataset.txt',
					help="학습할 데이터를 불러오는 경로입니다.")
parser.add_argument('--batch_size', type=int, default=8,
					help="batch_size 를 지정합니다.")
args = parser.parse_args()

pytorch_kogpt2 = {
    'url':
    'checkpoint/pytorch_kogpt2_676e9bcfa7.params',
    'fname': 'pytorch_kogpt2_676e9bcfa7.params',
    'chksum': '676e9bcfa7'
}

kogpt2_config = {
    "initializer_range": 0.02,
    "layer_norm_epsilon": 1e-05,
    "n_ctx": 1024,
    "n_embd": 768,
    "n_head": 12,
    "n_layer": 12,
    "n_positions": 1024,
    "vocab_size": 50000
}

def auto_enter(text):
	text = (text.replace("   ", "\n"))
	text = text.split("\n")

	text = [t.lstrip() for t in text if t != '']
	return "\n\n".join(text)

def get_gpu_memory_map():
	"""Get the current gpu usage.
	Returns
	-------
	usage: dict
		Keys are device ids as integers.
		Values are memory usage as integers in MB.
	"""
	result = subprocess.check_output(
		[
			'nvidia-smi', '--query-gpu=memory.used',
			'--format=csv,nounits,noheader'
		], encoding='utf-8')
	# Convert lines into a dictionary
	gpu_memory = [int(x) for x in result.strip().split('\n')]
	gpu_memory_map = dict(zip(range(len(gpu_memory)), gpu_memory))
	return gpu_memory_map

def main(epoch, save_path, load_path, samples, data_file_path, batch_size):
	ctx = 'cuda'
	cachedir = '~/kogpt2/'

	summary = SummaryWriter()

	# download model
	model_info = pytorch_kogpt2
	model_path = download(model_info['url'],
						   model_info['fname'],
						   model_info['chksum'],
						   cachedir=cachedir)
	# download vocab
	vocab_info = tokenizer
	vocab_path = download(vocab_info['url'],
						   vocab_info['fname'],
						   vocab_info['chksum'],
						   cachedir=cachedir)

	# KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
	kogpt2model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))

	# model_path 로부터 다운로드 받은 내용을 load_state_dict 으로 업로드
	kogpt2model.load_state_dict(torch.load(model_path))

	device = torch.device(ctx)
	kogpt2model.to(device)

	# 불러오기 부분
	try:
		checkpoint = torch.load(load_path, map_location=device)

		# KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
		kogpt2model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))
		kogpt2model.load_state_dict(checkpoint['model_state_dict'])

		kogpt2model.eval()
	except:
		count = 0
	else:
		count = int(re.findall("\d+", load_path)[1])

	print(count)
	# 추가로 학습하기 위해 .train() 사용
	kogpt2model.train()
	vocab_b_obj = gluonnlp.vocab.BERTVocab.from_sentencepiece(vocab_path,
														 mask_token=None,
														 sep_token=None,
														 cls_token=None,
														 unknown_token='<unk>',
														 padding_token='<pad>',
														 bos_token='<s>',
														 eos_token='</s>')


	tok_path = get_tokenizer()
	model, vocab = kogpt2model, vocab_b_obj
	tok = SentencepieceTokenizer(tok_path)

	dataset = Read_Dataset(data_file_path, vocab, tok)
	print("Read_Dataset ok")
	data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True)



	learning_rate = 3e-5
	criterion = torch.nn.CrossEntropyLoss()
	optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

	print('KoGPT-2 Transfer Learning Start')
	avg_loss = (0.0, 0.0)

	for epoch in range(epoch):
		for data in data_loader:
			optimizer.zero_grad()
			data = torch.stack(data) # list of Tensor로 구성되어 있기 때문에 list를 stack을 통해 변환해준다.
			data = data.transpose(1,0)
			data = data.to(ctx)
			model = model.to(ctx)

			outputs = model(data, labels=data)
			loss, logits = outputs[:2]
			loss = loss.to(ctx)
			loss.backward()
			avg_loss = (avg_loss[0] * 0.99 + loss, avg_loss[1] * 0.99 + 1.0)
			optimizer.step()
			if count % 10 == 0:
				print('epoch no.{0} train no.{1}  loss = {2:.5f} avg_loss = {3:.5f}' . format(epoch, count, loss, avg_loss[0] / avg_loss[1]))
				summary.add_scalar('loss/avg_loss', avg_loss[0] / avg_loss[1], count)
				summary.add_scalar('loss/loss', loss, count)

			# generator 진행
			if (count > 0 and count % 1000 == 0) or (len(data) < batch_size):
				sent = sample_sequence(model.to("cpu"), tok, vocab, sent="사랑", text_size=100, temperature=0.7, top_p=0.8, top_k=40)
				sent = sent.replace("<unused0>", "\n") # 비효율적이지만 엔터를 위해서 등장
				sent = auto_enter(sent)
				print(sent)

				summary.add_text('Text', sent, count)

				if count > 500000:
					now = [int(n) for n in os.listdir(samples)]
					now = max(now)
					f = open(samples + str(now + 1), 'w', encoding="utf-8")
					f.write(sent)
					f.close()
			#########################################
			count += 1

			if (count > 0 and count % 10000 == 0) or (len(data) < batch_size):
				# 모델 저장
				try:
					torch.save({
						'epoch': epoch,
						'train_no': count,
						'model_state_dict': model.state_dict(),
						'optimizer_state_dict': optimizer.state_dict(),
						'loss': loss
					}, save_path + 'KoGPT2_checkpoint_' + str(count) + '.tar')
				except:
					pass

if __name__ == "__main__":
	main(args.epoch, args.save_path, args.load_path, args.samples, args.data_file_path, args.batch_size)

In [5]:
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text)

In [9]:
input_ids

[33245, 10114, 12748, 11357]

In [12]:
generated

'근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.\n특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.\n또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.\n아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.\n운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.\n운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.\n운동을'

In [11]:
gen_ids = model.generate(torch.tensor([input_ids]),
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0,:].tolist())